# SVAE-based variant generation using pulldown assay data: LY6C1-Fc


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
import random
import tensorflow as tf
import os
import sys

from pathlib import Path
current_dir = Path(os.getcwd())
top_dir = current_dir.parent
sys.path.append(str(top_dir))


from svae.SVAE import SVAE
from svae.train import train
from svae.generate import generate_sequences
from utils.data_processing import prep_data, make_tf_batches, seq_to_onehot
from utils.plots import plot_latent_space


In [3]:
tf.__version__

'2.5.0'

## Data


In [4]:
df_all = pd.read_csv('../data/pulldown_assay_SVAE_data.csv')
df_all

,AA_sequence,starter_virus_mean_RPM,LY6A_log2enr,LY6A_cv_RPM,LY6A_mean_RPM,LY6C1_log2enr,LY6C1_cv_RPM,LY6C1_mean_RPM
0,NAVSRAG,333.473218,-3.201301,0.405151,36.255449,-1.724217,0.103689,100.929909
1,VRPNGSG,328.441120,-3.052515,0.254412,39.587581,-1.046010,0.142009,159.065919
2,GTRPGLL,239.470574,-4.047358,0.655096,14.483582,-3.680601,0.484687,18.675866
3,FGGSRTP,215.893749,-2.866125,0.392306,29.610816,-1.367466,0.287778,83.674266
4,LDKRNLV,202.031793,-3.827988,0.322309,14.225926,-2.289964,0.198510,41.311582
...,...,...,...,...,...,...,...,...
1750113,AARAATG,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1750114,GVPGYAW,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1750115,GNYLHAL,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1750116,REPSPSR,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000


In [5]:
# For this example, we'll look only at LY6C1 (analysis for LY6A is analogous)
df = df_all[['AA_sequence', 'starter_virus_mean_RPM', 'LY6C1_log2enr', 'LY6C1_cv_RPM', 'LY6C1_mean_RPM']]
df

,AA_sequence,starter_virus_mean_RPM,LY6C1_log2enr,LY6C1_cv_RPM,LY6C1_mean_RPM
0,NAVSRAG,333.473218,-1.724217,0.103689,100.929909
1,VRPNGSG,328.441120,-1.046010,0.142009,159.065919
2,GTRPGLL,239.470574,-3.680601,0.484687,18.675866
3,FGGSRTP,215.893749,-1.367466,0.287778,83.674266
4,LDKRNLV,202.031793,-2.289964,0.198510,41.311582
...,...,...,...,...,...
1750113,AARAATG,0.000000,NaN,NaN,0.000000
1750114,GVPGYAW,0.000000,NaN,NaN,0.000000
1750115,GNYLHAL,0.000000,NaN,NaN,0.000000
1750116,REPSPSR,0.000000,NaN,NaN,0.000000


### Drop variants with 0 starter and assay mean RPM


In [6]:
df = df[df['starter_virus_mean_RPM'] > 0][df['LY6C1_mean_RPM'] > 0].reset_index(drop=True)
df

/var/folders/h3/zgsh7pcs2jg1t0bvb6ddrhw80000gq/T/ipykernel_23573/3574525862.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['starter_virus_mean_RPM'] > 0][df['LY6C1_mean_RPM'] > 0].reset_index(drop=True)


,AA_sequence,starter_virus_mean_RPM,LY6C1_log2enr,LY6C1_cv_RPM,LY6C1_mean_RPM
0,NAVSRAG,333.473218,-1.724217,0.103689,100.929909
1,VRPNGSG,328.441120,-1.046010,0.142009,159.065919
2,GTRPGLL,239.470574,-3.680601,0.484687,18.675866
3,FGGSRTP,215.893749,-1.367466,0.287778,83.674266
4,LDKRNLV,202.031793,-2.289964,0.198510,41.311582
...,...,...,...,...,...
322965,DKNSQAC,0.042499,3.354674,1.732051,0.434743
322966,DKKASPW,0.042499,2.354674,1.732051,0.217372
322967,EFDHKSK,0.042499,2.354674,1.732051,0.217372
322968,EFASRYS,0.042499,2.191695,1.732051,0.194152


### Prep data and make Tensorflow-compatible data batches

`prep_data()` will one-hot encode the `AA_sequence` column and split the data into
training and test sets. The training split will be split
futher into training and validation batches with `make_tf_batches()`.


In [7]:
(data_list, train_df, test_df, prepped_df) = prep_data(df, target_assay_col="LY6C1_log2enr", cv_col="LY6C1_cv_RPM", RPM_col="LY6C1_mean_RPM")
train_df

,AA_sequence,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x132,x133,x134,x135,x136,x137,x138,x139,Y--LY6C1_log2enr,CV--LY6C1_cv_RPM
0,NAVSRAG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1.724217,0.103689
1,VRPNGSG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1.046010,0.142009
2,GTRPGLL,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,-3.680601,0.484687
3,FGGSRTP,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,-1.367466,0.287778
4,SRPGYPT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,-1.918459,0.018527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290668,EFGALRG,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.885911,1.732051
290669,DKKASPW,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2.354674,1.732051
290670,EFDHKSK,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.354674,1.732051
290671,EFASRYS,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2.191695,1.732051


In [8]:
# Pickle dataframes
dfs_dict = {'data_list': data_list, 'train_df': train_df, 'test_df': test_df, 'prepped_df': prepped_df}
Path('saved_vals/').mkdir(parents=True, exist_ok=True)
with open('saved_vals/dfs_dict.pickle', 'wb') as f:
    pickle.dump(dfs_dict, f)

In [9]:
# Load dataframes 
with open('saved_vals/dfs_dict.pickle', 'rb') as f:
    dfs_dict = pickle.load(f)

data_list = dfs_dict['data_list']
train_df = dfs_dict['train_df']
test_df = dfs_dict['test_df']
prepped_df = dfs_dict['prepped_df']

Now we split `train_df` into TensorFlow-compatible training and validation batches using
`make_tf_batches()`. `unbatched_data` is a tuple of Numpy arrays corresponding to the X, Y
and CV (if using) values across both the training and validation batches.


In [10]:
X, Y, CV = data_list
train_batches, val_batches, unbatched_data = make_tf_batches(X, Y, CV=CV)

2022-10-31 13:08:44.648847: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# Save batches to disk
tf.data.experimental.save(train_batches, '../saved_vals/train_batches')
tf.data.experimental.save(val_batches, '../saved_vals/val_batches')

2022-10-31 13:08:44.988251: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-31 13:08:55.006219: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 128664 of 322970
2022-10-31 13:09:05.004650: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 163690 of 322970
2022-10-31 13:09:15.006984: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 171388 of 322970
2022-10-31 13:09:25.005101: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 184793 of 322970
2022-10-31 13:09:35.005173: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 197585 of 322970
2022-10-31 13:09:45.005253: I tensorflow/core/kernels/data/shuffle_dataset

In [12]:
# Load Tensorflow batches
train_batches = tf.data.experimental.load('../saved_vals/train_batches')
val_batches = tf.data.experimental.load('../saved_vals/val_batches')

2022-10-31 13:17:48.593931: E tensorflow/core/framework/dataset.cc:825] Unimplemented: Cannot merge options for dataset of type LoadDataset, because the dataset does not implement `InputDatasets`.
2022-10-31 13:17:48.605622: E tensorflow/core/framework/dataset.cc:825] Unimplemented: Cannot merge options for dataset of type LoadDataset, because the dataset does not implement `InputDatasets`.


## SVAE training

Default parameterization (used in paper):

- `input_dim = 140`
- `latent_dim = 2`
- `enc_hidden_dims = [100,40]`
- `dec_hidden_dims = [40,100]`
- `reg_hidden_dims = [100,10]`


In [13]:
# SVAE parameters (same as default)
input_dim = 140
latent_dim = 2
enc_hidden_dims = [100,40]
dec_hidden_dims = [40,100]
reg_hidden_dims = [100,10]

# training parameters 
assay = 'LY6C1'
loss_weights = [1.0,0.5,0.1] # [reconstruction, KL, regression]

initial_learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(initial_learning_rate)

In [14]:
# Initialize SVAE model (with default parameterizations)
model = SVAE(input_dim=140, latent_dim=2, enc_hidden_dims=[100,40], dec_hidden_dims=[40,100], reg_hidden_dims=[100,10], name='svae')
model.summary()

Model: "svae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 140)]        0                                            
__________________________________________________________________________________________________
encoder (Functional)            [(None, 2), (None, 2 18304       encoder_input[0][0]              
                                                                 encoder_input[0][0]              
__________________________________________________________________________________________________
tf.concat_1 (TFOpLambda)        (None, 142)          0           encoder[1][0]                    
                                                                 encoder_input[0][0]              
_______________________________________________________________________________________________

The `train()` function returns the trained `model` and `preds_df`, a DataFrame of
regressor predictions on the X values in `unbatched_data`, along with their latent space
projections and their true Y values.

`train()` saves the trained model, as well as a log of the training and validation loss
values per epoch and `preds_df`, to disk. These saves will be made to an optional
parameter `model_outdir` - if not specified, `train()` will make a new,
informatively-named directory to write to (path is returned).

In [ ]:
# Train model
model, preds_df, model_outdir = train(model, train_batches, val_batches, train_df,
                        optimizer=optimizer,
                        loss_weights=loss_weights           
                        )

2022-10-31 13:17:48.953761: E tensorflow/core/framework/dataset.cc:825] Unimplemented: Cannot merge options for dataset of type LoadDataset, because the dataset does not implement `InputDatasets`.


Logging following metrics: ['train_model_loss', 'train_reconstruction_loss', 'train_kl_loss', 'train_regression_loss', 'val_model_loss', 'val_reconstruction_loss', 'val_kl_loss', 'val_regression_loss']


----- Beginning training. -----

epoch 1


2022-10-31 13:17:50.027416: E tensorflow/core/framework/dataset.cc:825] Unimplemented: Cannot merge options for dataset of type LoadDataset, because the dataset does not implement `InputDatasets`.


258336/258376 [============================>.] - ETA: 0s - train_model_loss: 19.4607 - train_reconstruction_loss: 19.3015 - train_kl_loss: 0.0090 - train_regression_loss: 3.1061

2022-10-31 13:25:23.050776: E tensorflow/core/framework/dataset.cc:825] Unimplemented: Cannot merge options for dataset of type LoadDataset, because the dataset does not implement `InputDatasets`.


...

258336/258376 [============================>.] - ETA: 0s - train_model_loss: 19.4019 - train_reconstruction_loss: 19.2976 - train_kl_loss: 0.0077 - train_regression_loss: 2.0178

2022-11-01 15:31:10.517157: E tensorflow/core/framework/dataset.cc:825] Unimplemented: Cannot merge options for dataset of type LoadDataset, because the dataset does not implement `InputDatasets`.


258376/258376 [==============================] - 545s 2ms/step - train_model_loss: 19.5331 - train_reconstruction_loss: 19.4209 - train_kl_loss: 0.0071 - train_regression_loss: 2.3440 - val_model_loss: 19.5536 - val_reconstruction_loss: 19.4250 - val_kl_loss: 0.0064 - val_regression_loss: 2.7020

epoch 61
Converged. Stall: 7/10
    64/258376 [..............................] - ETA: 12:58 - train_model_loss: 18.8780 - train_reconstruction_loss: 18.7285 - train_kl_loss: 0.0036 - train_regression_loss: 3.1913

2022-11-01 15:31:52.228598: E tensorflow/core/framework/dataset.cc:825] Unimplemented: Cannot merge options for dataset of type LoadDataset, because the dataset does not implement `InputDatasets`.


206432/258376 [======================>.......] - ETA: 2:13 - train_model_loss: 19.5259 - train_reconstruction_loss: 19.4053 - train_kl_loss: 0.0057 - train_regression_loss: 2.5071- ETA: 3:26 - tra - ETA: 3:25 - train_model_loss: 1 - ETA: 3:26 - train_model_loss: 19.5661 - train_reconstruction_loss: 19.4182 - train_kl_loss: 0.0047 - train_regression_loss: 2.9 - ETA: 3:26 - train_model_loss: 20.1276 - train_reconstruction_loss: 

### Plot latent space

In [ ]:
plots_outdir = model_outdir / 'plots'

fig, ax = plot_latent_space(preds_df, plots_outdir=plots_outdir, assay="LY6C1")

## Generate novel sequences

In [ ]:
sequences_outdir = model_outdir / 'sequences'
cluster_model_outdir = model_outdir / 'kmeans_models'
generated_sequences_dict, generated_sequences_list = generate_sequences(preds_df, 
    assay='LY6C1',
    show_plots=True,
    show_logos=False,
    show_subcluster_plots=True,
    show_sublogos=False,
    plots_outdir=plots_outdir,
    cluster_model_outdir=cluster_model_outdir,
    subcluster_model_outdir=cluster_model_outdir,
    sequences_outdir=sequences_outdir,
    AA_percentile_threshold=80
    )